<a href="https://colab.research.google.com/github/tsholofelo-mokheleli/ACIS-2023-New-Zealand/blob/main/Model_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas  as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
cmap=sns.color_palette('Blues_r')


from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
df = pd.read_csv('diabetes_binary_health_indicators_BRFSS2015.csv')
df.shape

(253680, 22)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_binary       253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [20]:
# Split by gender
female_df = df[df['Sex'] == 0]
male_df = df[df['Sex'] == 1]

In [21]:
# Define features (X) and target (y) for each gender
X_female = female_df.drop(columns=['Diabetes_binary'])
y_female = female_df['Diabetes_binary']

X_male = male_df.drop(columns=['Diabetes_binary'])
y_male = male_df['Diabetes_binary']

In [22]:
Xf_train, Xf_test, yf_train, yf_test = train_test_split(X_female, y_female, test_size=0.2, stratify=y_female, random_state=42)
Xm_train, Xm_test, ym_train, ym_test = train_test_split(X_male, y_male, test_size=0.2, stratify=y_male, random_state=42)

In [23]:
# Initialize SMOTETomek
smt = SMOTETomek(random_state=42)

# Apply to female data
Xf_resampled, yf_resampled = smt.fit_resample(Xf_train, yf_train)

# Apply to male data
Xm_resampled, ym_resampled = smt.fit_resample(Xm_train, ym_train)

# Check balance
print("Female Class Distribution After Resampling:")
print(pd.Series(yf_resampled).value_counts())

print("\nMale Class Distribution After Resampling:")
print(pd.Series(ym_resampled).value_counts())

Female Class Distribution After Resampling:
Diabetes_binary
0.0    98778
1.0    98778
Name: count, dtype: int64

Male Class Distribution After Resampling:
Diabetes_binary
0.0    75750
1.0    75750
Name: count, dtype: int64


In [24]:
rf_female = RandomForestClassifier(random_state=42)
rf_female.fit(Xf_resampled, yf_resampled)
y_pred_female = rf_female.predict(Xf_test)
print("Female Model Report:\n", classification_report(yf_test, y_pred_female))

rf_male = RandomForestClassifier(random_state=42)
rf_male.fit(Xm_resampled, ym_resampled)
y_pred_male = rf_male.predict(Xm_test)
print("Male Model Report:\n", classification_report(ym_test, y_pred_male))

Female Model Report:
               precision    recall  f1-score   support

         0.0       0.89      0.97      0.93     24713
         1.0       0.47      0.20      0.28      3682

    accuracy                           0.87     28395
   macro avg       0.68      0.58      0.60     28395
weighted avg       0.84      0.87      0.84     28395

Male Model Report:
               precision    recall  f1-score   support

         0.0       0.87      0.95      0.91     18955
         1.0       0.43      0.20      0.27      3387

    accuracy                           0.84     22342
   macro avg       0.65      0.58      0.59     22342
weighted avg       0.80      0.84      0.81     22342

